# NFL Playoffs Analysis
SQL queries to answer questions about the NFL Playoffs 2025. Created to get a better sense of the teams playing in the playoffs, and potentially give a 'prediction' of who might be moving forward to the Superbowl. 

In [0]:
%sql
-- Question #1 What are the win/loss rates of each team throughout the season?
-- Why? To give an overview of each team that is participating in the Playoffs

-- Aggregate win/loss records by team and game type
SELECT 
  full as team_full,
  team,
  -- Normalize game_type to 'POST' for playoffs, 'REG' for regular season
  CASE
    WHEN game_type = 'WC' OR game_type = 'DIV' THEN 'POST'
    WHEN game_type = 'REG' THEN 'REG'
  END as game_type,
  -- Count wins for each team
  SUM(CASE WHEN result = 'win' THEN 1 ELSE 0 END) as win,
  -- Count losses for each team
  SUM(CASE WHEN result = 'loss' THEN 1 ELSE 0 END) as loss
FROM (
    -- Away team results
    SELECT 
      away_team AS team,
      game_type,
      -- Determine win/loss for away team
      CASE
        WHEN CAST(away_score AS INT) > CAST(home_score AS INT) THEN 'win'
        WHEN CAST(home_score AS INT) > CAST(away_score AS INT) THEN 'loss'
      END as result
    FROM nfl_schedule_data_2025
    WHERE week <= 20 -- Just temporary restriction since data has not been updated for the entire season
      -- Playoff Teams
      AND away_team IN ('CHI','GB','BUF','LAR','JAX','DEN','CAR','HOU','NE','PHI','SF','SEA','LAC','PIT','LA')
    
    UNION ALL

    -- Home team results
    SELECT 
      home_team AS team,
      game_type,
      -- Determine win/loss for home team
      CASE
        WHEN CAST(home_score AS INT) > CAST(away_score AS INT) THEN 'win'
        WHEN CAST(away_score AS INT) > CAST(home_score AS INT) THEN 'loss'
      END as result
    FROM nfl_schedule_data_2025
    WHERE week <= 20 -- Just temporary restriction since data has not been updated for the entire season
      -- Playoff Teams
      AND home_team IN ('CHI','GB','BUF','LAR','JAX','DEN','CAR','HOU','NE','PHI','SF','SEA','LAC','PIT','LA')
) result_query
-- Joining team info to get full team names (for clarity) using USING for simplicity
INNER JOIN nfl_team_info_2025 USING (team)
GROUP BY team,game_type, full;

In [0]:
%sql
-- Question #2 What is each teams Offensive & Defensive yards overall and per game?
-- Why? To give an overview of each team's offensive and defensive capabilities. To give a potential indication of who might win against who in the next game/s.

-- CTE to filter stats for the playoff teams season data
WITH CTE as (
  SELECT 
		t1.season,
		t1.week,
		t1.season_type,
		t1.team,
		t1.opponent_team,
		t1.passing_yards,
		t1.sack_yards_lost,
		t1.rushing_yards
	FROM nfl_team_stats_2025 as t1
	-- Limit to only include the playoff teams
	WHERE t1.team IN ('CHI','GB','BUF','LAR','JAX','DEN','CAR','HOU','NE','PHI','SF','SEA','LAC','PIT','LA')
)

-- Join CTE to itself to get both teams' (team and opponent team) stats for each matchup, mainly to get the defensive yards allowed
SELECT
	t.season,
	t.week,
	t.season_type,
	t.team,
	ti.full as full_team_name,
	t.opponent_team,
  -- Calculate total yards (net passing + rushing)
  (t.passing_yards+t.sack_yards_lost) + t.rushing_yards AS total_yards,
  -- Calculate net passing yards (passing yards + sack yards lost)
  (t.passing_yards+t.sack_yards_lost) AS total_net_passing_yards,
  -- Rushing yards
  t.rushing_yards as total_rushing_yards,
	-- Defensive yards allowed (opponent's total yards)
	(t2.passing_yards+t2.sack_yards_lost) + t2.rushing_yards AS defensive_yards
FROM CTE AS t -- 'team 1/team'
JOIN CTE AS t2 -- 'team 2/opponent team'
	ON t.team = t2.opponent_team
	AND t.opponent_team = t2.team
	AND t.week = t2.week
	AND t.season = t2.season
-- Joining team info table for full team names (for clarity)
INNER JOIN nfl_team_info_2025 as ti
	ON t.team = ti.team
ORDER BY t.week;


In [0]:
%sql
-- Question #3 What is each teams QB's performance in passing yards throughout the season?
-- Why? To provide extra stats to back-up the teams performance, to see if the quarterbacks ability to throw the ball has a big impact.

SELECT
    -- Quarterback & general info
    player_name,
    player_display_name,
    position,
    week,
    season_type,
    t.team,
    ti.full as full_team_name,
    -- Quarterback stats
    (completions / attempts * 1.0) as completion_pct,
    -- Total games played
    COUNT(*) OVER (PARTITION BY week,t.team,player_id) as total_games_played,
    passing_yards,
    -- Total Passing yards per QB (for filtering purposes)
    SUM(passing_yards) OVER (PARTITION BY player_id) as total_passing_yards,
    completions,
    attempts
  FROM nfl_player_stats_2025 as t
  -- Joining on team info to get full team name (for clarity)
  INNER JOIN nfl_team_info_2025 as ti
  ON t.team = ti.team
  WHERE position = 'QB'
    -- Making sure the QBs included has been active in the season
    AND (completions > 1 OR attempts > 1)
    -- Playoff teams
    AND t.team IN ('CHI','GB','BUF','LAR','JAX','DEN','CAR','HOU','NE','PHI','SF','SEA','LAC','PIT','LA');

In [0]:
%sql
-- Question #4 What is the completion % of each QB throughout the season?
-- Why? To provide extra stats to back-up the teams performance, to see if the quarterbacks ability to throw the ball has a big impact.
-- Similar query to the Passing yards one, but just looking at the completion percentage. Only grouped by Player & Team

SELECT
    -- Quarterback & general info
    player_name,
    player_display_name,
    full_team_name,
    (SUM(completions) / SUM(attempts) * 1.0) as completion_pct,
    SUM(total_games_played) as total_games_played,
    SUM(completions) as total_completions,
    SUM(attempts) as total_attempts
FROM (
SELECT
    -- Quarterback & general info
    player_name,
    player_display_name,
    position,
    week,
    season_type,
    t.team,
    ti.full as full_team_name,
    -- Quarterback stats
    (completions / attempts * 1.0) as completion_pct,
    COUNT(*) OVER (PARTITION BY week,t.team,player_id) as total_games_played,
    passing_yards,
    completions,
    attempts
  FROM nfl_player_stats_2025 as t
  -- Joining on team info to get full team name (for clarity)
  INNER JOIN nfl_team_info_2025 as ti
  ON t.team = ti.team
  WHERE position = 'QB'
    -- Making sure the QBs included has been active in the season
    AND (completions > 1 OR attempts > 1)
    -- Playoff teams
    AND t.team IN ('CHI','GB','BUF','LAR','JAX','DEN','CAR','HOU','NE','PHI','SF','SEA','LAC','PIT','LA')
)
GROUP BY player_name,player_display_name, full_team_name;